In [ ]:
import requests
import importlib
import pandas as pd
from pathlib import Path
from collections import Counter

import brew_omics
importlib.reload(brew_omics)

# Really only experiments are needed for ENTEx and others are obtained from ENCODE folder
report_expt, report_expt_path = brew_omics.fetch_encode_metadata(
    "data/entex/report_expt.pkl",
    "https://www.encodeproject.org/report.tsv?type=Experiment&status=released&internal_tags=ENTEx&limit=all",
    check_ids=['Accession'], skip_rows=1, sep='\t', data_type='Experiment', overwrite=False)

# report_sam, report_sam_path = brew_omics.fetch_encode_metadata(
#     "data/entex/report_sam.pkl",
#     "https://www.encodeproject.org/report.tsv?type=Biosample&status=released&internal_tags=ENTEx&limit=all",
#     check_ids=['Accession'], skip_rows=1, sep='\t', data_type='Biosample', overwrite=False)

# metadata_file, metadata_file_path = brew_omics.fetch_encode_metadata(
#     "data/entex/metadata_file.pkl",
#     "https://www.encodeproject.org/metadata/?type=Experiment&status=released&internal_tags=ENTEx&limit=all",
#     check_ids=['File accession'], skip_rows=0, sep='\t', data_type='File', overwrite=False)

# Somehow this failed, probably internal_tags does not work for type=File
# report_file, report_file_path = brew_omics.fetch_encode_metadata(
#     "data/entex/report_file.pkl",
#     "https://www.encodeproject.org/report.tsv?type=File&status=released&internal_tags=ENTEx&limit=all",
#     check_ids=['ID', 'Dataset'],
#     skip_rows=1, sep='\t',
#     data_type='File')

Loaded cached ENCODE Experiment metadata with shape: (1586, 40)
Experiment metadata column 'Accession' value counts: Counter({'ENCSR940FNL': 1, 'ENCSR780CNW': 1, 'ENCSR642DSR': 1, 'ENCSR172LVU': 1, 'ENCSR860GPM': 1, 'ENCSR237GVU': 1, 'ENCSR011HZJ': 1, 'ENCSR181ATL': 1, 'ENCSR743GCE': 1, 'ENCSR525JJM': 1, 'ENCSR360UUM': 1, 'ENCSR153AXQ': 1, 'ENCSR276OPI': 1, 'ENCSR522MTS': 1, 'ENCSR753HPM': 1, 'ENCSR595MTV': 1, 'ENCSR965BLU': 1, 'ENCSR948TOS': 1, 'ENCSR701GIE': 1, 'ENCSR764OHK': 1, 'ENCSR545WFH': 1, 'ENCSR189JIH': 1, 'ENCSR274FJN': 1, 'ENCSR123ZFD': 1, 'ENCSR557QRO': 1, 'ENCSR326YHP': 1, 'ENCSR043XTM': 1, 'ENCSR258OVY': 1, 'ENCSR720WEB': 1, 'ENCSR080JPX': 1, 'ENCSR156WTZ': 1, 'ENCSR514EZP': 1, 'ENCSR589DBF': 1, 'ENCSR621MTP': 1, 'ENCSR074WIB': 1, 'ENCSR714FDJ': 1, 'ENCSR429VWL': 1, 'ENCSR948DNK': 1, 'ENCSR152HLZ': 1, 'ENCSR868ZEI': 1, 'ENCSR757EGB': 1, 'ENCSR652WJE': 1, 'ENCSR178FVP': 1, 'ENCSR012KZW': 1, 'ENCSR845BEG': 1, 'ENCSR803MVC': 1, 'ENCSR585KBH': 1, 'ENCSR799KLF': 1, 'ENCSR702O

In [15]:
# Get the master file report with biosample information
encode_file_biosample = pd.read_pickle("data/encode/report_file_biosample.pkl")
encode_file_biosample.set_index('Accession', inplace=True, drop=False)
encode_file_biosample.index.name = 'Index'
encode_file_accs = set(encode_file_biosample['Accession'].tolist())

# column "Files" in report_expt contains list of file ids, separated by commas
entex_file_accs = []
for file_list in report_expt['Files'].dropna():
    entex_file_accs.extend([f.strip('/').split('/')[-1] for f in file_list.split(',')])

print(f"A total of file accessions found in experiments: {len(entex_file_accs)}")
print(entex_file_accs[:5])

files_not_found = [f for f in entex_file_accs if f not in encode_file_accs]
if len(files_not_found) == 0:
    print("All files in experiments are present in encode_file_biosample.")
else:
    print(f"Missing files in encodes_file_biosample: {files_not_found}")

# Extract the rows in report_file_biosample corresponding to the file_accessions
report_file_biosample = encode_file_biosample.loc[entex_file_accs]
print(f"Extracted report_file_biosample_entex shape: {report_file_biosample.shape}")

# Save the extracted report_file_biosample_entex
report_file_biosample.to_pickle("data/entex/report_file_biosample.pkl")
report_file_biosample.to_csv("data/entex/report_file_biosample.csv", index=False)
print("Saved extracted report_file_biosample_entex to data/entex/report_file_biosample_entex.pkl/csv")

# save files_with_biosample for future use
file_with_biosample = report_file_biosample[report_file_biosample['Biosample accession'].notna()]
print(f"file_with_biosample shape: {file_with_biosample.shape}")
file_with_biosample.to_pickle("data/entex/file_with_biosample.pkl")
file_with_biosample.to_csv("data/entex/file_with_biosample.csv", index=False)
print("Saved file_with_biosample to data/entex/file_with_biosample.pkl/csv")

A total of file accessions found in experiments: 37291
['ENCFF854PQD', 'ENCFF851TIV', 'ENCFF565HBG', 'ENCFF542ZZT', 'ENCFF064KQN']
All files in experiments are present in encode_file_biosample.
Extracted report_file_biosample_entex shape: (37291, 90)
Saved extracted report_file_biosample_entex to data/entex/report_file_biosample_entex.pkl/csv
file_with_biosample shape: (5462, 90)
Saved file_with_biosample to data/entex/file_with_biosample.pkl/csv


In [16]:
report_sam_merged = pd.read_pickle("data/encode/report_sam_merged.pkl")
print(f"Loaded ENCODE biosample data with shape: {report_sam_merged.shape}")
report_sam_merged.set_index('Accession', inplace=True, drop=False)
report_sam_merged.index.name = 'Index'

final_biosample_list = file_with_biosample['Biosample accession'].unique().tolist()
print(f"Number of unique biosample accessions in files_with_biosample: {len(final_biosample_list)}")

# Get the list of biosample accessions from files_with_biosample
final_biosamples = report_sam_merged[report_sam_merged['Accession'].isin(final_biosample_list)]
final_biosamples.to_pickle("data/entex/final_biosamples.pkl")
final_biosamples.to_csv("data/entex/final_biosamples.csv", index=False)
print("Saved final filtered biosample data to data/entex/final_biosamples.pkl/csv")

Loaded ENCODE biosample data with shape: (29930, 34)
Number of unique biosample accessions in files_with_biosample: 669
Saved final filtered biosample data to data/entex/final_biosamples.pkl/csv


In [ ]:
importlib.reload(brew_omics)
download_path = Path(f"data/entex/download")

assay_file_types = {
    'RNA-seq': ['fastq',],
    'DNase-seq': ['fastq',],
}

report_file_biosample.set_index('Accession', inplace=True, drop=False)
report_file_biosample.index.name = 'Index'

files_groupby_biosample = report_file_biosample.groupby('Biosample accession')
files_groupby_dataset = report_file_biosample.groupby('Dataset')

importlib.reload(brew_omics)
saved_file_accessions = brew_omics.download_encode_files_groupby_biosample(
    report_file_biosample,
    biosample_accessions=list(files_groupby_biosample.groups.keys()),
    assay_file_types=assay_file_types,
    download_path=download_path,
    files_groupby_biosample=files_groupby_biosample,
    files_groupby_dataset=files_groupby_dataset,
)

print(f"Downloaded a total of {len(saved_file_accessions)} files for selected biosamples and assay types.")
final_files_downloaded = report_file_biosample[report_file_biosample['Accession'].isin(saved_file_accessions)]
final_files_downloaded.to_pickle(download_path / "final_files_downloaded.pkl")
final_files_downloaded.to_csv(download_path / "final_files_downloaded.csv", index=False)
print(f"Saved final downloaded files metadata to {download_path / 'final_files_downloaded.pkl'}/{download_path / 'final_files_downloaded.csv'}")

Biosamples:   4%|▍         | 28/669 [00:00<00:02, 276.98it/s]

Biosample: ENCBS002CZJ; saved metadata to data/entex/dataset/ENCBS002CZJ.csv
    Assay: RNA-seq, Datasets: []
    Assay: DNase-seq, Datasets: []
Biosample: ENCBS004OVW; saved metadata to data/entex/dataset/ENCBS004OVW.csv
    Assay: RNA-seq, Datasets: []
    Assay: DNase-seq, Datasets: []
Biosample: ENCBS005MNT; saved metadata to data/entex/dataset/ENCBS005MNT.csv
    Assay: RNA-seq, Datasets: []
    Assay: DNase-seq, Datasets: []
Biosample: ENCBS005NEU; saved metadata to data/entex/dataset/ENCBS005NEU.csv
    Assay: RNA-seq, Datasets: []
    Assay: DNase-seq, Datasets: []
Biosample: ENCBS006KAW; saved metadata to data/entex/dataset/ENCBS006KAW.csv
    Assay: RNA-seq, Datasets: ['/experiments/ENCSR754WLW/', '/experiments/ENCSR264COY/']
    Dataset /experiments/ENCSR754WLW/ has 27 files.
        Excluding 3 'transcriptome alignments' bam files for RNA-seq assay.
    File accession: ENCFF306MRG
        Checking derived from accession: /files/ENCFF107OTF/ for file accession: ENCFF306MRG
 

Biosamples:  13%|█▎        | 84/669 [00:00<00:02, 275.30it/s]

Biosample: ENCBS085LTC; saved metadata to data/entex/dataset/ENCBS085LTC.csv
    Assay: RNA-seq, Datasets: ['/experiments/ENCSR471RUK/', '/experiments/ENCSR418FVT/']
    Dataset /experiments/ENCSR471RUK/ has 27 files.
        Excluding 3 'transcriptome alignments' bam files for RNA-seq assay.
    File accession: ENCFF178ZZZ
        Checking derived from accession: /files/ENCFF582ILA/ for file accession: ENCFF178ZZZ
        File accession: ENCFF582ILA is directly from biosample: ENCBS085LTC
    File accession: ENCFF879VGW
        Checking derived from accession: /files/ENCFF582ILA/ for file accession: ENCFF879VGW
        File accession: ENCFF582ILA is directly from biosample: ENCBS085LTC
    File accession: ENCFF857JQM
        Checking derived from accession: /files/ENCFF582ILA/ for file accession: ENCFF857JQM
        File accession: ENCFF582ILA is directly from biosample: ENCBS085LTC
    Dataset /experiments/ENCSR418FVT/ has 13 files.
    File accession: ENCFF867DIK
        Checking de

Biosamples:  17%|█▋        | 112/669 [00:00<00:02, 249.24it/s]

...

Biosample: ENCBS801NKM; saved metadata to data/entex/dataset/ENCBS801NKM.csv
    Assay: RNA-seq, Datasets: []
    Assay: DNase-seq, Datasets: []
Biosample: ENCBS802IML; saved metadata to data/entex/dataset/ENCBS802IML.csv
    Assay: RNA-seq, Datasets: []
    Assay: DNase-seq, Datasets: []
Biosample: ENCBS805AZN; saved metadata to data/entex/dataset/ENCBS805AZN.csv
    Assay: RNA-seq, Datasets: []
    Assay: DNase-seq, Datasets: []
Biosample: ENCBS806WZK; saved metadata to data/entex/dataset/ENCBS806WZK.csv
    Assay: RNA-seq, Datasets: []
    Assay: DNase-seq, Datasets: []
Biosample: ENCBS812XPB; saved metadata to data/entex/dataset/ENCBS812XPB.csv
    Assay: RNA-seq, Datasets: []
    Assay: DNase-seq, Datasets: []
Biosample: ENCBS812ZNH; saved metadata to data/entex/dataset/ENCBS812ZNH.csv
    Assay: RNA-seq, Datasets: ['/experiments/ENCSR485WBR/', '/experiments/ENCSR269YSX/']
    Dataset /experiments/ENCSR485WBR/ has 27 files.
        Excluding 3 'transcriptome alignments' bam files 

Biosamples:  97%|█████████▋| 652/669 [00:02<00:00, 269.31it/s]

Biosample: ENCBS892ESJ; saved metadata to data/entex/dataset/ENCBS892ESJ.csv
    Assay: RNA-seq, Datasets: []
    Assay: DNase-seq, Datasets: []
Biosample: ENCBS894VJJ; saved metadata to data/entex/dataset/ENCBS894VJJ.csv
    Assay: RNA-seq, Datasets: ['/experiments/ENCSR812AKX/', '/experiments/ENCSR968DMR/']
    Dataset /experiments/ENCSR812AKX/ has 27 files.
        Excluding 3 'transcriptome alignments' bam files for RNA-seq assay.
    File accession: ENCFF087LVI
        Checking derived from accession: /files/ENCFF663VCC/ for file accession: ENCFF087LVI
        File accession: ENCFF663VCC is directly from biosample: ENCBS894VJJ
    File accession: ENCFF772DXL
        Checking derived from accession: /files/ENCFF663VCC/ for file accession: ENCFF772DXL
        File accession: ENCFF663VCC is directly from biosample: ENCBS894VJJ
    File accession: ENCFF171EQY
        Checking derived from accession: /files/ENCFF598IDH/ for file accession: ENCFF171EQY
        File accession: ENCFF598ID

Biosamples: 100%|██████████| 669/669 [00:02<00:00, 243.34it/s]


Biosample: ENCBS974IWX; saved metadata to data/entex/dataset/ENCBS974IWX.csv
    Assay: RNA-seq, Datasets: []
    Assay: DNase-seq, Datasets: ['/experiments/ENCSR164WOF/']
    Dataset /experiments/ENCSR164WOF/ has 40 files.
    File accession: ENCFF909FQD
        Checking derived from accession: /files/ENCFF762MJQ/ for file accession: ENCFF909FQD
        File accession: ENCFF762MJQ not found in report_file_biosample index.
        Checking derived from accession: /files/ENCFF258TNQ/ for file accession: ENCFF909FQD
        Checking derived from accession: /files/ENCFF418NRL/ for file accession: ENCFF258TNQ
        File accession: ENCFF418NRL is directly from biosample: ENCBS974IWX
    File accession: ENCFF628FMV
        Checking derived from accession: /files/ENCFF496RSW/ for file accession: ENCFF628FMV
        File accession: ENCFF496RSW not found in report_file_biosample index.
        Checking derived from accession: /files/ENCFF987DVS/ for file accession: ENCFF628FMV
        File ac

In [ ]:
# generate inputs to spliser from report_file_biosample and final_biosamples

download_path = Path(f"data/entex/dataset")

assay_file_types = {
    'RNA-seq': ['bam',],
    'DNase-seq': ['bam',],
}
genome_assembly = 'GRCh38'
genome_annotation = 'V29' # not used currently

unique_classifications = report_file_biosample['Biosample classification'].unique().tolist()
print(f"Unique biosample classifications: {unique_classifications}")
# assert all([cls.lower() == 'tissue' for cls in unique_classifications]), "Not all biosample classifications are 'tissue'"

files_groupby_tissue = report_file_biosample.groupby('Biosample term name')
print(f"Number of unique biosample term names: {len(files_groupby_tissue.groups)}, e.g., {list(files_groupby_tissue.groups.keys())[:5]}")

# For each sample type, find samples with a given assay type, and then get a list of bam files from all samples
for tissue, group in files_groupby_tissue:
    print(f"Processing sample type: {tissue} with {len(group)} files")
    for assay_type, file_exts in assay_file_types.items():
        # get the list of Biosample accessions for this tissue and assay type
        biosample_accessions = group[group['Assay term name'].str.lower() == assay_type.lower()]['Biosample accession'].unique().tolist()

        for file_type in file_exts:
            print(f"  Assay type: {assay_type}, file type: {file_type}, number of biosamples: {len(biosample_accessions)}")
            tissue_assay_files = []

            for biosample_acc in biosample_accessions:
                file_path = download_path / biosample_acc / assay_type 
                # find files of this type for this biosample under the path
                files_of_type = list(file_path.glob(f"*.{file_type}"))
                print(f"    Biosample accession: {biosample_acc}, found {len(files_of_type)} files of type {file_type}")

                # Todo: check the file size and md5sum to ensure the file is correctly downloaded
                for f in files_of_type:
                    file_acc = f.stem
                    # check the Genome assembly of the file
                    file_metadata = report_file_biosample.loc[file_acc]
                    file_genome_assembly = file_metadata['Genome assembly']
                    if file_genome_assembly != genome_assembly:
                        print(f"      Skipping file {file_acc} due to genome assembly mismatch: {file_genome_assembly} != {genome_assembly}")
                        continue
                    
                    print(f"      Adding file {file_acc} with genome assembly {file_genome_assembly}")
                    tissue_assay_files.append(str(f))
                
            if tissue_assay_files:
                output_file = download_path / f"{tissue.replace(' ', '_')}_{assay_type}_{file_type}_files.txt"
                with open(output_file, 'w') as f:
                    for bam_file in tissue_assay_files:
                        f.write(f"{bam_file}\n")
                print(f"    Saved file list to: {output_file}")
            else:
                print(f"    No files found for tissue {tissue} and assay {assay_type}.")
    break

Unique biosample classifications: [nan, 'tissue']
Number of unique biosample term names: 31, e.g., ["Peyer's patch", 'adrenal gland', 'ascending aorta', 'body of pancreas', 'breast epithelium']
Processing sample type: Peyer's patch with 188 files
  Assay type: RNA-seq, file type: bam, number of biosamples: 4
    Biosample accession: ENCBS231DCI, found 5 files of type bam
      Adding file ENCFF199FHX with genome assembly GRCh38
      Skipping file ENCFF475PRY due to genome assembly mismatch: hg19 != GRCh38
      Adding file ENCFF112DAY with genome assembly GRCh38
      Adding file ENCFF980MFI with genome assembly GRCh38
      Skipping file ENCFF158VOW due to genome assembly mismatch: hg19 != GRCh38
    Biosample accession: ENCBS943XLR, found 5 files of type bam
      Adding file ENCFF202RSI with genome assembly GRCh38
      Skipping file ENCFF998TZQ due to genome assembly mismatch: hg19 != GRCh38
      Adding file ENCFF814SRA with genome assembly GRCh38
      Skipping file ENCFF402RCG 